In [2]:
import numpy as np
import os
import pickle
import copy 
import json
import warnings
warnings.filterwarnings("ignore")
from load_img import Load_from_Folder, Load_Images
from evaluate import Time, MSE, PSNR
from LANCZOS import *
from GIC import GIC

def downsample(X, r):
    a, b =  LANCZOS.split(X, r)   
    return a

def run1():
    n_img=-1
    par_old1 = {'n_hop': 6, 
     'win': {'hop1': [256, 128, 64, 32, 16, 8], 
             'hop2': [128, 64, 32, 16, 8, 4], 
             'hop3': [64, 32, 16, 8, 4], 
             'hop4': [32, 16, 8, 4], 
             'hop5': [16, 8, 4], 
             'hop6': [8, 4]}, 
     "MSE_TH": {"hop1": [-1, 70, 70, 70, 70, 70], 
                   "hop2": [-1, 70, 70, 70, 70, 70], 
                   "hop3": [-1, 70, 70, 70, 70], 
                   "hop4": [-1, 70, 70, 70], 
                   "hop5": [-1, 70,70], 
                   "hop6": [-1, 70]}, 
     'n_clusters': {'hop1': [64, 128, 512, 512, 512, 64], 
                    'hop2': [64, 128, 512, 512, 512, 128], 
                    'hop3': [64, 128, 512, 512, 512], 
                    'hop4': [64, 128, 512, 512], 
                    'hop5': [64, 32, 64], 
                    'hop6': [64, 32]}, 
     'shape': {'hop6': [[-1, 1, 1, 1], [-1, 2, 2, 1]], 
               'hop5': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1]], 
               'hop4': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1]], 
               'hop3': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1]], 
               'hop2': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1], [-1, 32, 32, 1]], 
               'hop1': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1], [-1, 32, 32, 1]]}, 
     'n_components': {'hop1': [150, 150, 150, 50, 30, 20, None], 
                      'hop2': [150, 150, 50, 30, 20, None], 
                      'hop3': [100, 40, 20, 12, None], 
                      'hop4': [40, 20, 12, None], 
                      'hop5': [20, 12, None], 
                      'hop6': [12, None]}, 'gpu': {'hop1': [False, False, False, False, False, False, False, False, False], 'hop2': [False, False, False, False, False, False, False, False], 'hop3': [False, False, False, False, False, False, False], 'hop4': [False, False, False, False, False, False], 'hop5': [False, False, False, False, False], 'hop6': [False, False, False, False]}}
    
    print('--------------------start---------------------')
    print(par_old1)
    with open('dataset.json', 'r') as json_file:
        DataSet = json.load(json_file)
    Y = Load_from_Folder(folder=DataSet['CLIC_train_1024x1024_more'], color='RGB', ct=-1)+\
        Load_from_Folder(folder=DataSet['Holopix50k_trainR_1024x1024'], color='RGB', ct=2000)
    Y = np.array(Y).astype('float32')
    Y = downsample(Y, 4)
    print(Y.shape)
    gic = GIC(par_old1)
    gic.fit(Y)
    return gic

In [3]:
gic = run1()

--------------------start---------------------
{'n_hop': 6, 'win': {'hop1': [256, 128, 64, 32, 16, 8], 'hop2': [128, 64, 32, 16, 8, 4], 'hop3': [64, 32, 16, 8, 4], 'hop4': [32, 16, 8, 4], 'hop5': [16, 8, 4], 'hop6': [8, 4]}, 'MSE_TH': {'hop1': [-1, 70, 70, 70, 70, 70], 'hop2': [-1, 70, 70, 70, 70, 70], 'hop3': [-1, 70, 70, 70, 70], 'hop4': [-1, 70, 70, 70], 'hop5': [-1, 70, 70], 'hop6': [-1, 70]}, 'n_clusters': {'hop1': [64, 128, 512, 512, 512, 64], 'hop2': [64, 128, 512, 512, 512, 128], 'hop3': [64, 128, 512, 512, 512], 'hop4': [64, 128, 512, 512], 'hop5': [64, 32, 64], 'hop6': [64, 32]}, 'shape': {'hop6': [[-1, 1, 1, 1], [-1, 2, 2, 1]], 'hop5': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1]], 'hop4': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1]], 'hop3': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1]], 'hop2': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1], [-1, 32, 32, 1]], 'hop1': [[-1, 1, 1, 1], [-1, 2, 2, 1]

In [4]:

from color_space import *
x = cv2.imread('lena512color.tiff')
x = BGR2RGB(x)
x = x.reshape(1, 512, 512, -1).astype('float32')
Y = downsample(x, 2)
stream, dc, save = gic.encode(Y)
iY = gic.decode(stream, dc)
print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))


   <RunTime> foreward:  0.0 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.0 s
hop4
   <RunTime> backward:  0.0 s
hop3
   <RunTime> backward:  0.0 s
hop2
   <RunTime> backward:  0.1 s
hop1

Testing MSE: 144.8162257771897
Hop-1
   save tree:0.00404358
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00009155 bpp
  Level-2 Huffman 0.00076294 bpp
  Level-3 Huffman 0.00511169 bpp
  Level-4 Huffman 0.02131653 bpp
  Level-5 Huffman 0.04943848 bpp
Hop-2
   save tree:0.00483704
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00015259 bpp
  Level-2 Huffman 0.00123596 bpp
  Level-3 Huffman 0.00729370 bpp
  Level-4 Huffman 0.02903748 bpp
  Level-5 Huffman 0.07304382 bpp
Hop-3
   save tree:0.00129700
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00016785 bpp
  Level-2 Huffman 0.00169373 bpp
  Level-3 Huffman 0.00828552 bpp
  Level-4 Huffman 0.03227234 bpp
Hop-4
   save tree:0.00032043
  Level-0 Huffman 0.00007629 bpp
  Level-1 Huffman 0.00030518 bpp
  Level-2 H

In [5]:
with open('dataset.json', 'r') as json_file:
    DataSet = json.load(json_file)
Y = Load_from_Folder(folder=DataSet['CLIC_test_1024x1024_more'], color='RGB', ct=-1)
Y = np.array(Y).astype('float32')
Y = downsample(Y, 4)
stream, dc, save = gic.encode(Y)
iY = gic.decode(stream, dc)
print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))
print('--------------------end---------------------\n\n\n')

   <RunTime> foreward:  3.7 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.1 s
hop4
   <RunTime> backward:  0.5 s
hop3
   <RunTime> backward:  1.7 s
hop2
   <RunTime> backward:  6.5 s
hop1

Testing MSE: 236.72874544958134
Hop-1
   save tree:0.00360189
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00010090 bpp
  Level-2 Huffman 0.00065096 bpp
  Level-3 Huffman 0.00469766 bpp
  Level-4 Huffman 0.02018672 bpp
  Level-5 Huffman 0.05083170 bpp
Hop-2
   save tree:0.00382438
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00015735 bpp
  Level-2 Huffman 0.00111775 bpp
  Level-3 Huffman 0.00587020 bpp
  Level-4 Huffman 0.02325202 bpp
  Level-5 Huffman 0.06404762 bpp
Hop-3
   save tree:0.00112144
  Level-0 Huffman 0.00002871 bpp
  Level-1 Huffman 0.00018261 bpp
  Level-2 Huffman 0.00152801 bpp
  Level-3 Huffman 0.00693561 bpp
  Level-4 Huffman 0.02551360 bpp
Hop-4
   save tree:0.00030632
  Level-0 Huffman 0.00003667 bpp
  Level-1 Huffman 0.00029049 bpp
  Level-2 

In [6]:
for k in gic.MBMBVQ.km.keys():
    km = gic.MBMBVQ.km[k]
    for i in km.KM:
        i.KM.KM = None
        i.KM.saveObj=False
with open('/mnt/yifan/gic_256-0918-1.pkl', 'wb') as f:
    pickle.dump(gic, f, 4)

In [8]:
refit = {"hop1": [(False, 0),(False, 0), (False, 0), (False, 0), (False, 0),(False, 8)], 
       "hop2": [(False, 0), (False, 0), (False, 0), (False, 0), (False, 0), (False, 0)], 
       "hop3": [(False, 0), (False, 0), (False, 0), (False, 0), (False, 0)], 
       "hop4": [(False, 0), (False, 0), (False, 0), (False, 0)], 
       "hop5": [(False, 0), (False, 0), (False, 0)], 
       "hop6": [(False, 0), (False, 0)]}
with open('dataset.json', 'r') as json_file:
    DataSet = json.load(json_file)
Y_train = Load_from_Folder(folder=DataSet['CLIC_train_1024x1024_more'], color='RGB', ct=-1)+\
        Load_from_Folder(folder=DataSet['Holopix50k_trainR_1024x1024'], color='RGB', ct=2000)
a = [8, 16,32, 64, 128]
tmp = {}
tmp1 = {}
for i in a:
    print(i,'----------------------------------')
    
    Y = np.array(copy.deepcopy(Y_train)).astype('float32')
    Y = downsample(Y, 4)
    refit['hop1'][-1] = (True, i)
    gic.refit(Y, refit)
    with open('/mnt/yifan/gic_256_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump(gic, f, 4)
    Y = None
    from color_space import *
    x = cv2.imread('lena512color.tiff')
    x = BGR2RGB(x)
    
    Y = x.reshape(1, 512, 512, -1).astype('float32')
    Y = downsample(Y, 2)
    stream, dc, save = gic.encode(Y)
    iY = gic.decode(stream, dc)
    print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))
    #'''
    tmp[i] = iY
    Y = None
    print('----------------------------------------------------------------------')
    Y = Load_from_Folder(folder=DataSet['CLIC_test_1024x1024_more'], color='YUV', ct=-1)
    Y = np.array(Y).astype('float32')
    Y = downsample(Y, 4)
    stream, dc, save = gic.encode(Y)
    iY = gic.decode(stream, dc)
    tmp1[i] = iY
    print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))
    print('--------------------end---------------------\n\n\n')
    #'''
    
with open('/mnt/yifan/lena_256.pkl','wb') as f:
    pickle.dump(tmp,f)
    
with open('/mnt/yifan/clic_test_256.pkl','wb') as f:
    pickle.dump(tmp,f,4)

8 ----------------------------------
   <RunTime> foreward: 47.8 s
hop6
MBVQ channel 3
   <RunTime> rebackward:  0.9 s
hop5
MBVQ channel 3
   <RunTime> rebackward:  2.7 s
hop4
MBVQ channel 3
   <RunTime> rebackward:  7.7 s
hop3
MBVQ channel 3
   <RunTime> rebackward: 32.8 s
hop2
MBVQ channel 3
   <RunTime> rebackward: 119.2 s
hop1
MBVQ channel 3
   MBVQ refit 5 8

Training MSE: 173.0634647289448
   <RunTime> refit: 454.5 s
   <RunTime> foreward: 46.3 s
hop6
   <RunTime> backward:  0.8 s
hop5
   <RunTime> backward:  1.7 s
hop4
   <RunTime> backward:  5.1 s
hop3
   <RunTime> backward: 19.3 s
hop2
   <RunTime> backward: 74.9 s
hop1

Testing MSE: 173.0634647289448
   <RunTime> refit: 785.5 s
   <RunTime> foreward:  0.0 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.0 s
hop4
   <RunTime> backward:  0.0 s
hop3
   <RunTime> backward:  0.0 s
hop2
   <RunTime> backward:  0.1 s
hop1

Testing MSE: 156.77715098788906
Hop-1
   save tree:0.00404358
  Level-0 Huffman 0.00001526 b

   <RunTime> foreward:  2.3 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.1 s
hop4
   <RunTime> backward:  0.3 s
hop3
   <RunTime> backward:  1.0 s
hop2
   <RunTime> backward:  3.7 s
hop1

Testing MSE: 200.45589331350686
Hop-1
   save tree:0.00353659
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00009803 bpp
  Level-2 Huffman 0.00063488 bpp
  Level-3 Huffman 0.00446631 bpp
  Level-4 Huffman 0.01929999 bpp
  Level-5 Huffman 0.03990559 bpp
Hop-2
   save tree:0.00374726
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00016325 bpp
  Level-2 Huffman 0.00108731 bpp
  Level-3 Huffman 0.00562852 bpp
  Level-4 Huffman 0.02235913 bpp
  Level-5 Huffman 0.06098536 bpp
Hop-3
   save tree:0.00110339
  Level-0 Huffman 0.00002502 bpp
  Level-1 Huffman 0.00018237 bpp
  Level-2 Huffman 0.00146296 bpp
  Level-3 Huffman 0.00672182 bpp
  Level-4 Huffman 0.02455935 bpp
Hop-4
   save tree:0.00030403
  Level-0 Huffman 0.00003552 bpp
  Level-1 Huffman 0.00028040 bpp
  Level-2 

   <RunTime> decode:  7.2 s
MSE=189.485, PSNR=25.35505
--------------------end---------------------



